## 1. init

In [ ]:
import re
from enum import Enum
from datetime import datetime, timedelta

import httpx
import getpass
import feedparser

In [ ]:
import json
from pprint import pprint

from IPython import display


def tofile(data):
    with open('./tmp.json', 'w') as f:
        json.dump(data, f)

In [ ]:
jira_host = 'https://jira.yovole.tech'

org_datepattern = "%Y-%m-%dT%H:%M:%S.%fZ"
dest_datepattern = "%Y-%m-%d"
UA = 'Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1'


class JiraApi(Enum):
    login = "/rest/mobile/1.0/login"
    activity = '/activity'
    issue_detail = "/rest/api/latest/issue/{issue_id}?expand=schema,names,transitions"

In [ ]:
def get_jira_client(username, password):
    client = httpx.Client(base_url=jira_host, verify=False, headers={'User-Agent': UA})
    login_data = {
        'os_username': username,
        'os_password': password,
        'os_cookie': 'true'
    }
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    login_resp = client.post(JiraApi.login.value, data=login_data, headers=headers)
    assert login_resp.status_code == 200, f"login error: {login_resp}"
    return client

## 2. login

In [ ]:
username = getpass.getpass(prompt='username: ')
client = get_jira_client(username, getpass.getpass(prompt='password: '))

## 3. jira detail

In [ ]:
params = {'expand': 'renderedFields,schema,changelog'}
issue_detail = client.get(JiraApi.issue_detail.value.format(issue_id='CMP2-00000'), params=params)
issue_detail_json = issue_detail.json()
display.display_json(issue_detail_json, raw=True)

if issue_detail.status_code == 200:
    res = {
        'id': issue_detail_json['key'],
        'priority': issue_detail_json['fields']['priority']['name'],
        'plan_start': issue_detail_json['fields']['customfield_13805'],
        'plan_end': issue_detail_json['fields']['customfield_13806'],
        # 'actual': issue,
        'reporter': issue_detail_json['fields']['reporter']['displayName'],
        'assignee': issue_detail_json['fields']['assignee']['displayName'],
        'title': issue_detail_json['fields']['summary'],
        'description': issue_detail_json['fields']['description'],
    }
    display.display_json(res, raw=True)

## 4. recent

In [ ]:
activity_user = username

In [ ]:
params = {'maxResults': 100, 'streams': 'user IS {}'.format(activity_user)}
resp_activity = client.get(JiraApi.activity.value, params=params)

In [ ]:
# a = '''

# '''
# feed_data = feedparser.parse(a)

In [ ]:
feed_data = feedparser.parse(resp_activity.text)

In [ ]:
# feed_data

In [ ]:
week_start = datetime(year=2024, month=1, day=22)

In [ ]:
activities_lis = []
entry_pattern = re.compile(r'/a.*?将.*?a href=".*?browse/(?P<id>.*?)">(?P<title>.*?)</a>.*?更新为(?P<status>.*?)$', re.S)
title_tag_pattern = re.compile('.*?(<span.*?>(.*)</span>)')
for entry in feed_data['entries']:
    searched = entry_pattern.search(entry['title'])
    if not searched:
        continue
    update_date = datetime.strptime(entry['updated'], org_datepattern) + timedelta(hours=8)
    if update_date >= week_start:
        data = {**searched.groupdict(), 'date': update_date.strftime(dest_datepattern)}
        title = data['title']
        tag_matched = title and title_tag_pattern.match(title)
        if tag_matched:
            data['title'] = f'{title[0:tag_matched.start(1)]}{tag_matched.group(2)}{title[tag_matched.end(1):]}'
        activities_lis.append(data)

### 基本信息 - 无开始结束时间

In [ ]:
id_set = set()
activities_distinct = []
for item in activities_lis:
    if item["id"] not in id_set:
        id_set.add(item["id"])
        activities_distinct.append(item)

In [ ]:
for item in activities_distinct:
    # print(f"- {item['id']}: {item['title']}")
    print(f"{item['title']}")

### 开始结束时间

In [ ]:
# filter data: 取最新的一条
data_filterd = []
keyset = set()
for d in activities_lis:
    key = d['id'] + d['status']
    if key not in keyset:
        keyset.add(key)
        data_filterd.append(d)
data_filterd

In [ ]:
# 根据 status 解析日期
for item in data_filterd:
    if item['status'] == '进行中':
        item['actual_start'] = item.pop('date')
    elif item['status'] == 'In Review':
        item['actual_end'] = item.pop('date')


In [ ]:
data_filterd